In [1]:
import numpy as np
import cv2

In [2]:
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)


In [3]:
lk_params=dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [4]:
cap = cv2.VideoCapture(0)

# Parameters for corner detection
corner_track_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the first frame
ret, prev_frame = cap.read()

if not ret:
    print("Failed to grab the frame")
    cap.release()
    cv2.destroyAllWindows()
else:
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Points to track
    prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

    # Create a mask for drawing purposes
    mask = np.zeros_like(prev_frame)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Convert the current frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow
        nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prev_points, None, **lk_params)

        # Select good points
        good_new = nextPts[status == 1]
        good_prev = prev_points[status == 1]

        # Draw the tracks
        for i, (new, prev) in enumerate(zip(good_new, good_prev)):
            x_new, y_new = new.ravel()
            x_prev, y_prev = prev.ravel()

            # Convert the coordinates to integers
            x_new, y_new = int(x_new), int(y_new)
            x_prev, y_prev = int(x_prev), int(y_prev)

            # Draw the line on the mask
            mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)

            # Draw the circle on the frame
            frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)

        # Combine the frame and the mask
        img = cv2.add(frame, mask)
        cv2.imshow('tracking', img)

        # Exit loop if the ESC key is pressed
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

        # Update previous frame and points
        prev_gray = frame_gray.copy()
        prev_points = good_new.reshape(-1, 1, 2)

    cv2.destroyAllWindows()
    cap.release()


KeyboardInterrupt: 

: 

DENSE opticalflowfunction

In [4]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam or provide a video file path as argument

# Read the first frame from the video
ret, frame1 = cap.read()

# Convert the frame to grayscale
prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Create a mask for HSV image
hsv_mask = np.zeros_like(frame1)
hsv_mask[:, :, 1] = 255  # Set the saturation to 255

while True:
    # Read the next frame from the video
    ret, frame2 = cap.read()
    
    if not ret:
        print("Failed to capture frame")
        break
    
    # Convert the frame to grayscale
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # Compute magnitude and angle of the flow vectors
    mag, ang = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees=True)
    
    # Set hue according to the flow direction
    hsv_mask[:, :, 0] = ang / 2
    
    # Set value according to the flow magnitude
    hsv_mask[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    
    # Convert HSV to BGR (for visualization)
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    
    # Display the frame with optical flow
    cv2.imshow('Optical Flow', bgr)
    
    # Exit if 'ESC' key is pressed
    if cv2.waitKey(30) & 0xff == 27:
        break
    
    # Update previous image for next iteration
    prvsImg = nextImg

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
